In [1]:
import $file.common
import common._
import doobie._, doobie.implicits._
import cats._, cats.implicits._
import cats.effect._
import fs2.Stream
import java.sql._
import doobie.hikari._

Compiling /Users/jserrano/Documents/tagless-final-tutorial/session1/common.sc

import $file.$     

import common._

import doobie._, doobie.implicits._

import cats._, cats.implicits._

import cats.effect._

import fs2.Stream

import java.sql._

import doobie.hikari._

# Variation 5. DAO in Scala

This is not DAO-based code. No interfaces for now.

Common case classes for DAO interfaces.

In [2]:
case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

defined class Country
defined class City

### IO

In [3]:
trait CityDAO{
    def city(id: Int): IO[City]
    def cityName(id: Int): IO[String]
    def cityPopulation(id: Int): IO[Int]
    def cityCountryCode(id: Int): IO[String]
}

trait CountryDAO{
    def country(code: String): IO[Country]
    def countryName(code: String): IO[String]
    def countryCapital(code: String): IO[Option[Int]]
}

trait World extends CityDAO with CountryDAO{
    def countries(): IO[List[String]]
    def cities(): IO[List[Int]]
}

defined trait CityDAO
defined trait CountryDAO
defined trait World

In [4]:
val IOListM = implicitly[Monad[λ[T => IO[List[T]]]]]
val IOListF = implicitly[FunctorFilter[λ[T => IO[List[T]]]]](FunctorFilterM[IO, List])    

IOListM: Monad[IO[List[T]]] = ammonite.$sess.common$Helper$$anon$1@26105385
IOListF: FunctorFilter[IO[List[T]]] = ammonite.$sess.common$Helper$$anon$2@6b470e72

In [5]:
// No stream fusion
// Can't use for-comprehensions (need a monad for λ[T => IO[List[T]]]) :(
// Needs implicit conversions from IO[T]/IO[Option[T]] to IO[List[T]]

def largeCapitalsIOList(W: World): IO[List[(String, String)]] =
    IOListM.flatMap(W.countries){ countryCode => 
        IOListM.flatMap(
            IOListF.filter(W.country(countryCode))(_.capital.isDefined)
        ){ country => 
            IOListM.map(
                IOListF.filter(W.city(country.capital.get))(_.population > 8000000)
            ){ city => 
                (country.name, city.name)
            }
        }
    }

defined function largeCapitalsIOList

In [6]:
class DoobieWorldIOList(xa: Transactor[IO]) extends World{
    def city(id: Int): IO[City] = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].unique.transact(xa)

    def cityName(id: Int): IO[String] = 
        sql"select name from city where id = $id"
            .query[String].unique.transact(xa)
    
    def cityPopulation(id: Int): IO[Int] = 
        sql"select population from city where id = $id"
            .query[Int].unique.transact(xa)
    
    def cityCountryCode(id: Int): IO[String] = 
        sql"select countryCode from city where id = $id"
            .query[String].unique.transact(xa)

    def country(code: String): IO[Country] = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].unique.transact(xa)
    
    def countryName(code: String): IO[String] =
        sql"select name from country where code = $code"
            .query[String].unique.transact(xa)
    
    def countryCapital(code: String): IO[Option[Int]] =
        sql"select capital from country where code = $code"
            .query[Int].option.transact(xa)
    
    def countries(): IO[List[String]] = 
        sql"select code from country"
            .query[String].to[List].transact(xa)
    
    def cities(): IO[List[Int]] = 
        sql"select id from city"
            .query[Int].to[List].transact(xa)
}

object DoobieWorldIOList extends DoobieWorldIOList(xa)

defined class DoobieWorldIOList
defined object DoobieWorldIOList

In [7]:
largeCapitalsIOList(DoobieWorldIOList).unsafeRunSync.timed

6041 millis


res6: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

BIG BIG BIG PROBLEM: no filters AND query avalance AND multiple connections

### with connection pools

In [8]:
val htransactor: Resource[IO, HikariTransactor[IO]] =
    for {
        ce <- ExecutionContexts.fixedThreadPool[IO](32) // our connect EC
        be <- Blocker[IO]    // our blocking EC
        xa <- HikariTransactor.newHikariTransactor[IO](
              "org.postgresql.Driver",                        // driver classname
              "jdbc:postgresql:world",   // connect URL
              "postgres",                                   // username
              "",                                     // password
              ce,                                     // await connection here
              be                                      // execute JDBC operations here
            )
    } yield xa

htransactor: Resource[IO, HikariTransactor[IO]] = Bind(
  Bind(
    Allocate(
      Map(
        Delay(doobie.util.ExecutionContexts$$$Lambda$3546/1034856150@4c233a74),
        scala.Function1$$Lambda$318/1443055846@36d1e270,
        1
      )
    ),
    cats.effect.Resource$$Lambda$3413/107602644@4ca6aa63
  ),
  ammonite.$sess.cmd7$Helper$$Lambda$3549/1640704476@64880817
)

In [9]:
htransactor.map(xa => new DoobieWorldIOList(xa)).use(largeCapitalsIOList).unsafeRunSync.timed

1449 millis


res8: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

### with single connection

In [10]:
val DoobieWorldIOListSingle = for {
    con <- Resource.make(
        IO(DriverManager.getConnection("jdbc:postgresql:world", "postgres", "")))(
        connection => IO(connection.close))
    be <- Blocker[IO]
} yield new DoobieWorldIOList(Transactor.fromConnection[IO](con, be))

DoobieWorldIOListSingle: Resource[IO, DoobieWorldIOList] = Bind(
  Allocate(
    Map(
      Delay(ammonite.$sess.cmd9$Helper$$Lambda$3598/1997694337@7491fd96),
      scala.Function1$$Lambda$318/1443055846@6c38bdbf,
      1
    )
  ),
  ammonite.$sess.cmd9$Helper$$Lambda$3600/1853654297@42a40ed5
)

In [11]:
DoobieWorldIOListSingle.use(largeCapitalsIOList).unsafeRunSync.timed

910 millis


res10: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

### `Stream[ConnectionIO, ?]`

This API is JDBC-specific. TODO: Stream[IO, ?]

In [12]:
trait CityDAOS{
    def city(id: Int): Stream[ConnectionIO, City]
    def cityName(id: Int): Stream[ConnectionIO, String]
    def cityPopulation(id: Int): Stream[ConnectionIO, Int]
    def cityCountryCode(id: Int): Stream[ConnectionIO, String]
}

trait CountryDAOS{
    def country(code: String): Stream[ConnectionIO, Country]
    def countryName(code: String): Stream[ConnectionIO, String]
    def countryCapital(code: String): Stream[ConnectionIO, Option[Int]]
}

trait WorldS extends CityDAOS with CountryDAOS{
    def countries(): Stream[ConnectionIO, String]
    def cities(): Stream[ConnectionIO, Int]
}

defined trait CityDAOS
defined trait CountryDAOS
defined trait WorldS

In [13]:
object WorldS extends WorldS {
    
    def city(id: Int): Stream[ConnectionIO, City] = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].stream
    
    def cityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def cityPopulation(id: Int): Stream[ConnectionIO, Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream

    def cityCountryCode(id: Int): Stream[ConnectionIO, String] = 
        sql"select population from city where id = $id"
            .query[String].stream

    def country(code: String): Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].stream
    
    def countryName(id: String): Stream[ConnectionIO, String] =
        sql"select name from country where id = $id"
            .query[String].stream
    
    def countryCapital(id: String): Stream[ConnectionIO, Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Option[Int]].stream
    
    def countries: Stream[ConnectionIO, String] = 
        sql"select code from country"
            .query[String].stream
    
    def cities: Stream[ConnectionIO, Int] = 
        sql"select code, name, capital from country"
            .query[Int].stream

}

defined object WorldS

In [14]:
def largeCapitalsStream(W: WorldS): Stream[ConnectionIO,(String, String)] = for {
    name <- W.countries
    Country(_, name, Some(capital)) <- W.country(name)
    population <- W.cityPopulation(capital) if population > 8000000
    cName <- W.cityName(capital)
} yield (cName, name)

defined function largeCapitalsStream

In [17]:
largeCapitalsStream(WorldS).compile
    .toList
    .transact(xa)
    .unsafeRunSync
    .timed

373 millis


res16: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

BIG PROBLEM: query avalanche (although single connection).